In [5]:
import numpy as np
from scipy.optimize import minimize

def isda_discount_factor(rates, time_points, mean_reversion, volatility):
    # Calculate the discount factor using the Hull-White model
    discount_factors = np.zeros_like(rates)
    discount_factors[0]=1

    for t in range(1, len(time_points)):
        dt = time_points[t] - time_points[t-1]
        sigma_dt = volatility * np.sqrt(dt)
        a = mean_reversion

        # Define the function for optimization
        objective_function = lambda x: (x * (1 - np.exp(-a * dt)) / a - np.exp(-x * dt) * discount_factors[t-1]) ** 2

        # Use optimization to find the optimal rate for each tenor
        result = minimize(objective_function, rates[t-1], bounds=[(0, 1)])

        discount_factors[t] = np.exp(-result.x * dt) * discount_factors[t-1]

    return discount_factors

def isda_hazard_rate(cds_spread, discount_factors, recovery_rate, notional, maturity):
    # Calculate the implied hazard rate using the CDS spread and discount factors
    protection_leg_pv = notional * cds_spread * (1 - recovery_rate) * (1 - discount_factors[-1])
    premium_leg_pv = notional * (1 - recovery_rate) * discount_factors[-1]

    hazard_rate = -np.log(1 - protection_leg_pv / premium_leg_pv) / maturity

    return hazard_rate

def isda_cds_pricing(notional, cds_spread, maturity, recovery_rate, market_rates, time_points, mean_reversion, volatility):
    # Perform curve bootstrapping to get discount factors
    discount_factors = isda_discount_factor(market_rates, time_points, mean_reversion, volatility)

    # Calculate the implied hazard rate using CDS spread and discount factors
    hazard_rate = isda_hazard_rate(cds_spread, discount_factors, recovery_rate, notional, maturity)

    # Calculate present value of protection leg (premiums)
    protection_leg_pv = notional * cds_spread * (1 - recovery_rate) * (1 - discount_factors[-1])

    # Calculate present value of premium leg (default payments)
    premium_leg_pv = notional * (1 - recovery_rate) * discount_factors[-1]

    # Calculate CDS price
    cds_price = protection_leg_pv - premium_leg_pv

    return cds_price, discount_factors, hazard_rate

# Example market rates for curve bootstrapping
market_rates = np.array([0.02, 0.025, 0.03, 0.035, 0.04]*252)

# Example parameters
notional = 1000000  # Notional amount
cds_spread = 0.02   # CDS spread (annualized)
maturity = 5         # Maturity in years
recovery_rate = 0.4  # Recovery rate in case of default
mean_reversion = 0.1
volatility = 0.01

# Time parameters
num_time_steps = 252 * maturity  # assuming daily time steps
time_points = np.linspace(0, maturity, num_time_steps)

# Calculate CDS price, discount factor, and hazard rate
cds_price, discount_factors, hazard_rate = isda_cds_pricing(
    notional, cds_spread, maturity, recovery_rate, market_rates, time_points, mean_reversion, volatility
)

#print

print(f"CDS Price: {cds_price:.6f}")
print(f"Implied Hazard Rate: {hazard_rate:.6%}")


C:\Users\n.kouakou\AppData\Local\Temp\ipykernel_73896\755850282.py:20: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  discount_factors[t] = np.exp(-result.x * dt) * discount_factors[t-1]


CDS Price: 7876.376437
Implied Hazard Rate: nan%


C:\Users\n.kouakou\AppData\Local\Temp\ipykernel_73896\755850282.py:29: RuntimeWarning: invalid value encountered in log
  hazard_rate = -np.log(1 - protection_leg_pv / premium_leg_pv) / maturity
